In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np

sys.path.append('../src')
import dem

In [5]:
np.cos(45), np.cos(np.radians(45))

(0.5253219888177297, 0.7071067811865476)

In [4]:
bounding_box = (-123.0, 37.0, -121.0, 38.0) 
dem.srtm(bounding_box, 'srtm_subset.tif')

Failed to download data


HTTPError: 401 Client Error: 401 for url: https://portal.opentopography.org/API/globaldem?demtype=SRTMGL1&outputFormat=GTiff&south=37.0&north=38.0&west=-123.0&east=-121.0